In [1]:
import os
from dotenv import load_dotenv
import json
import time
import web3

load_dotenv()

True

In [2]:
w3 = web3.Web3(web3.Web3.HTTPProvider("https://api.helium.fhenix.zone/"))
w3.eth.account.enable_unaudited_hdwallet_features()
acc = w3.eth.account.from_mnemonic(os.getenv("MNEMONIC"))

In [3]:
with open(
    "/Users/kosunghun/codes/projects/fugazi-core-ethonline2024/deployments/testnet/FugaziCore.json",
    "r",
) as file:
    FugaziCore_json = json.load(file)
with open(
    "/Users/kosunghun/codes/projects/fugazi-core-ethonline2024/deployments/testnet/FugaziViewerFacet.json",
    "r",
) as file:
    FugaziViewer_json = json.load(file)
with open(
    "/Users/kosunghun/codes/projects/fugazi-core-ethonline2024/deployments/testnet/FugaziPoolActionFacet.json",
    "r",
) as file:
    FugaziPoolAction_json = json.load(file)

FugaziViewer = w3.eth.contract(
    address=FugaziCore_json["address"], abi=FugaziViewer_json["abi"]
)
FugaziPoolAction = w3.eth.contract(
    address=FugaziCore_json["address"], abi=FugaziPoolAction_json["abi"]
)

In [4]:
totalUnclaimedOrders = FugaziViewer.functions.getUnclaimedProtocolOrdersLength().call()
print(f"Total outstanding protocol-owned account's orders are: {totalUnclaimedOrders}")

oldestUnclaimedOrder = FugaziViewer.functions.getUnclaimedProtocolOrder(0).call()
print(
    f"Oldest unclaimed order by protocol-owned account: \nPool Id: {oldestUnclaimedOrder[0].hex()}\nEpoch: {oldestUnclaimedOrder[1]}"
)

poolInfo = FugaziViewer.functions.getPoolInfo(oldestUnclaimedOrder[0]).call()
print(f"Pool's state: \nEpoch: {poolInfo[0]} \nLast Settlement: {poolInfo[1]}")

Total outstanding protocol-owned account's orders are: 1
Oldest unclaimed order by protocol-owned account: 
Pool Id: 20c6a1487f7b228d3eb292c560a5112481371a30bdb1a7584d240bb9fbdaba3d
Epoch: 1
Pool's state: 
Epoch: 2 
Last Settlement: 1725543875


In [6]:
try:
    w3.eth.send_raw_transaction(
        acc.sign_transaction(
            FugaziPoolAction.functions.claimProtocolOrder(
                FugaziViewer.functions.getUnclaimedProtocolOrder(0).call()[0],
                FugaziViewer.functions.getUnclaimedProtocolOrder(0).call()[1],
            ).build_transaction(
                {
                    "from": acc.address,
                    "nonce": w3.eth.get_transaction_count(acc.address),
                }
            )
        ).rawTransaction
    )
except Exception as e:
    print(e)

{'code': 3, 'message': 'execution reverted', 'data': '0x5fdb34e8'}
